In [ ]:
import matplotlib
%matplotlib inline

from matplotlib import rc
rc("font", family="serif", size=12)
rc("text", usetex=True)

import daft

# Instantiate the PGM.
pgm = daft.PGM([5, 6], origin=[0, 0])

#desired hyperparameters
pgm.add_node(daft.Node("cosmology", r"$\vec{\theta}$", 2.5, 5.5))

#latent variables/parameters
pgm.add_node(daft.Node("distance", r"$\mu_{n,m}$", 1.5, 4))
pgm.add_node(daft.Node("redshift", r"$z_{n,m}$", 3.5, 4))

#data
pgm.add_node(daft.Node("lightcurve", r"$\underline{\ell}_{n,m}$", 1.5, 3, observed=True))
pgm.add_node(daft.Node("photometry", r"$\vec{f}_{n,m}$", 3.5, 3, observed=True))

#latent variables/parameters
pgm.add_node(daft.Node("type", r"$T_{n}$", 2.5, 2))

#nuisance hyperparameters
pgm.add_node(daft.Node("types", r"$R_{m}$", 2.5, 1))

# Add in the edges.
pgm.add_edge("cosmology", "distance")
pgm.add_edge("cosmology", "redshift")
pgm.add_edge("distance", "lightcurve")
pgm.add_edge("distance", "photometry")
pgm.add_edge("redshift", "lightcurve")
pgm.add_edge("redshift", "photometry")
pgm.add_edge("types", "type")
pgm.add_edge("type", "lightcurve")

# plates
pgm.add_plate(daft.Plate([0.5, 0.5, 4, 4.5], label=r"$m = 1, \cdots, M$"))
pgm.add_plate(daft.Plate([1, 1.5, 3, 3], label=r"$n = 1, \cdots, N$"))

# Render and save.
pgm.render()
pgm.figure.savefig("Hubble-draft.png", dpi=150)
pgm.figure.show()